# Build the DC Dataset for Modeling
I'm used this notebook to build a file with the appropriate code mappings.  It is a convenience for the EDA and modeling notebooks.

The file is 1.8GB and is not included.

In [40]:
#https://ffiec.cfpb.gov/data-publication/snapshot-national-loan-level-dataset

import pandas as pd

hmdaLAR = pd.read_csv('C:/Data/2017_public_lar.txt',sep='|',header=None)

hmdaLAR.columns = [
    "As of Year",
    "Respondent ID",
    "Agency Code",
    "Loan Type",
    "Property Type",
    "Loan Purpose",
    "Occupancy",
    "Loan Amount (000s)",
    "Preapproval",
    "Action Type",
    "MSA/MD",
    "State Code",
    "County Code",
    "Census Tract Number",
    "Applicant Ethnicity",
    "Co Applicant Ethnicity",
    "Applicant Race 1",
    "Applicant Race 2",
    "Applicant Race 3",
    "Applicant Race 4",
    "Applicant Race 5",
    "Co Applicant Race 1",
    "Co Applicant Race 2",
    "Co Applicant Race 3",
    "Co Applicant Race 4",
    "Co Applicant Race 5",
    "Applicant Sex",
    "Co Applicant Sex",
    "Applicant Income (000s)",
    "Purchaser Type",
    "Denial Reason 1",
    "Denial Reason 2",
    "Denial Reason 3",
    "Rate Spread",
    "HOEPA Status",
    "Lien Status",
    "Edit Status",
    "Sequence Number",
    "Population",
    "Minority Population %",
    "FFIEC Median Family Income",
    "Tract to MSA/MD Income %",
    "Number of Owner-occupied units",
    "Number of 1-to 4-Family units",
    "Application Date Indicator"
]

### Create Origination Count By Respondent File

In [41]:
# filter to just originations
originations = hmdaLAR[hmdaLAR["Action Type"]==1]

# count originations by respondent
originations_by_respondent = originations["Respondent ID"].value_counts()
applications_by_respondent = hmdaLAR["Respondent ID"].value_counts()

# write file
originations_by_respondent.to_csv("2017 Originations By Respondent.csv",sep="|",index=True)

In [69]:
result = pd.concat([applications_by_respondent, originations_by_respondent], axis=1, sort=False)
result.columns=["Applications","Originations"]
result["2018 Exempt"] = result["Originations"] < 500

print("Total 2017 Applications:",result["Applications"].sum())
print("Total 2017 Reporters:",len(result.index))
print()
print("2018 Exemptions")
print(result[result["2018 Exempt"]].sum())

1021450/14285496

Total 2017 Applications: 14285496
Total 2017 Reporters: 5762

2018 Exemptions
Applications    1021450.0
Originations     630408.0
2018 Exempt        4268.0
dtype: float64


0.07150259255961432

### Filter By State (if needed)

In [25]:
# filter to DC
hmdaLAR = hmdaLAR[hmdaLAR["State Code"] == 11]


### Code Mapping

In [26]:
# mappings taken from code sheet provided by ffiec
# https://ffiec.cfpb.gov/data-publication/snapshot-national-loan-level-dataset

agencyCodeMap = pd.Series(
    ['Office of the Comptroller of the Currency (OCC)',
    'Federal Reserve System (FRS)',
    'Federal Deposit Insurance Corporation (FDIC)',
    'National Credit Union Administration (NCUA)',
    'Department of Housing and Urban Development (HUD)',
    'Consumer Financial Protection Bureau (CFPB)'], 
    index=[1,2,3,5,7,9])

loanTypeMap = pd.Series(
    [
        'Conventional (any loan other than FHA, VA, FSA, or RHS loans)',
        'FHA-insured (Federal Housing Administration)',
        'VA-guaranteed (Veterans Administration)',
        'FSA/RHS (Farm Service Agency or Rural Housing Service)'
    ],
    index=[1,2,3,4])

propertyTypeMap = pd.Series(
    [
        'One to four-family (other than manufactured housing)',
        'Manufactured housing',
        'Multifamily'
    ],
    index=[1,2,3])

loanPurposeMap = pd.Series(
    [
        'Home purchase',
        'Home improvement',
        'Refinancing'
    ],
    index=[1,2,3])

occupancyMap = pd.Series(
    [
        'Owner-occupied as a principal dwelling',
        'Not owner-occupied',
        None
    ],
    index=[1,2,3])

preapprovalMap = pd.Series(
    [
        'Preapproval was requested',
        'Preapproval was not requested',
        None
    ],
    index=[1,2,3])

actionTakenMap = pd.Series(
    [
        'Approved',
        'Approved',
        'Denied',
        None,
        None,
        'Approved',
        'Denied',
        'Approved',
    ],
    index=[1,2,3,4,5,6,7,8])

ethnicityMap = pd.Series(
    [
        'Hispanic or Latino',
        'Not Hispanic or Latino',
        None,
        None,
        None
    ],
    index=[1,2,3,4,5])

raceMap = pd.Series(
    [
        'American Indian or Alaska Native',
        'Asian',
        'Black or African American',
        'Native Hawaiian or Other Pacific Islander',
        'White',
        None,
        None,
        'No co-applicant'
    ],
    index=[1,2,3,4,5,6,7,8])

sexMap = pd.Series(
    [
        'Male',
        'Female',
        None,
        None,
        'No co-applicant'
    ],
    index=[1,2,3,4,5])

purchaserTypeMap = pd.Series(
    [
        'Loan was not originated or was not sold in calendar year covered by register',
        'Fannie Mae (FNMA)',
        'Ginnie Mae (GNMA)',
        'Freddie Mac (FHLMC)',
        'Farmer Mac (FAMC)',
        'Private securitization',
        'Commercial bank, savings bank or savings association',
        'Life insurance company, credit union, mortgage bank, or finance company',
        'Affiliate institution',
        'Other type of purchaser'
    ],
    index=[0,1,2,3,4,5,6,7,8,9])

denialReasonMap = pd.Series(
    [
        'Debt-to-income ratio',
        'Employment history',
        'Credit history',
        'Collateral',
        'Insufficient cash (down payment, closing costs)',
        'Unverifiable information',
        'Credit application incomplete',
        'Mortgage insurance denied',
        'Other'
    ],
    index=[1,2,3,4,5,6,7,8,9])

hoepaMap = pd.Series(
    [
        'HOEPA loan',
        'Not a HOEPA loan'
    ],
    index=[1,2])

lienMap = pd.Series(
    [
        'Secured by a first lien',
        'Secured by a subordinate lien',
        'Not secured by a lien',
        'Not applicable (purchased loans)'
    ],
    index=[1,2,3,4])

hmdaLAR["Action Type"].replace(actionTakenMap,inplace=True)
hmdaLAR["Agency Code"].replace(agencyCodeMap,inplace=True)
hmdaLAR["Loan Type"].replace(loanTypeMap,inplace=True)
hmdaLAR["Property Type"].replace(propertyTypeMap,inplace=True)
hmdaLAR["Loan Purpose"].replace(loanPurposeMap,inplace=True)
hmdaLAR["Occupancy"].replace(occupancyMap,inplace=True)
hmdaLAR["Preapproval"].replace(preapprovalMap,inplace=True)
hmdaLAR["Action Type"].replace(actionTakenMap,inplace=True)
hmdaLAR["Applicant Ethnicity"].replace(ethnicityMap,inplace=True)
hmdaLAR["Co Applicant Ethnicity"].replace(ethnicityMap,inplace=True)
hmdaLAR["Applicant Race 1"].replace(raceMap,inplace=True)
hmdaLAR["Applicant Race 2"].replace(raceMap,inplace=True)
hmdaLAR["Applicant Race 3"].replace(raceMap,inplace=True)
hmdaLAR["Applicant Race 4"].replace(raceMap,inplace=True)
hmdaLAR["Applicant Race 5"].replace(raceMap,inplace=True)
hmdaLAR["Co Applicant Race 1"].replace(raceMap,inplace=True)
hmdaLAR["Co Applicant Race 2"].replace(raceMap,inplace=True)
hmdaLAR["Co Applicant Race 3"].replace(raceMap,inplace=True)
hmdaLAR["Co Applicant Race 4"].replace(raceMap,inplace=True)
hmdaLAR["Co Applicant Race 5"].replace(raceMap,inplace=True)
hmdaLAR["Applicant Sex"].replace(sexMap,inplace=True)
hmdaLAR["Co Applicant Sex"].replace(sexMap,inplace=True)
hmdaLAR["Purchaser Type"].replace(purchaserTypeMap,inplace=True)
hmdaLAR["Denial Reason 1"].replace(denialReasonMap,inplace=True)
hmdaLAR["Denial Reason 2"].replace(denialReasonMap,inplace=True)
hmdaLAR["Denial Reason 3"].replace(denialReasonMap,inplace=True)
hmdaLAR["HOEPA Status"].replace(hoepaMap,inplace=True)
hmdaLAR["Lien Status"].replace(lienMap,inplace=True)

### Add Calculated Fields

In [27]:
hmdaLAR["Loan To Income Ratio"] = hmdaLAR["Loan Amount (000s)"]/hmdaLAR["Applicant Income (000s)"]

originations_by_respondent = pd.read_csv("2017 Originations By Respondent.csv",sep="|")
originations_by_respondent.columns = ["Respondent ID","Origination Count"]

hmdaLAR = hmdaLAR.merge(originations_by_respondent, left_on='Respondent ID', right_on='Respondent ID', how='outer')
hmdaLAR["Small Institution"] = hmdaLAR["Origination Count"] < 500

### Write File

In [31]:
#hmdaLAR.to_csv("2017 DC LAR.csv",sep="|",index=False)